In [ ]:
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dropout


Using TensorFlow backend.


In [ ]:
# split a univariate dataset into train/test sets
def split_dataset(data, window):
	# split into standard weeks
	train, test = data[:(data.shape[0]-window*2),], data[(data.shape[0]-window*2):-window,]
	# restructure into windows of weekly data
	train_array = array(split(train, train.shape[0]/window))
	test_array = array(split(test, test.shape[0]/window))
	return train_array, test_array

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=1):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(data.shape[0]):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= data.shape[0]:
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train, n_input):
  trainx, trainy = to_supervised(train, n_input)
  trainy = trainy.reshape((trainy.shape[0], trainy.shape[1],))
  window_size = n_input
  num_features = train.shape[2]
  output_dim =1  #num_features
  model = Sequential()
  model.add(LSTM(100, activation = 'relu', input_shape=(window_size, num_features), return_sequences=True))
  model.add(LSTM(64, activation = 'relu', return_sequences = True))
  model.add(LSTM(32,activation='relu'))
  model.add(Dropout(rate=0.001))
  model.add(Dense(output_dim, activation='sigmoid'))
  model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae', 'mape', 'accuracy'])
  model.summary()
  model.fit(trainx,trainy, epochs=100, batch_size=20, verbose=2, shuffle=False)
  # history = model.fit(X,Y, epochs=50, batch_size=3, validation_split=0.2, verbose=2, shuffle=False)
  # save model
  model.save('FL_COVID-19_Model.h5')
 # model.fit(trainx, trainy, epochs=100, batch_size=batch_size, verbose=verbose)
  return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(test.shape[0]):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

In [ ]:
# load the new file
dataset = pd.read_csv('FL Dataset.csv', header=0)
dataset = pd.DataFrame(dataset,  columns = ['total_confirmed','total_deceased',
                                           #'total_recovered',
                                           'total_tested', 'mobility_retail_and_recreation',
                                           'mobility_grocery_and_pharmacy', 'mobility_parks',
                                           'mobility_transit_stations', 'mobility_workplaces',
                                           'mobility_residential', 'average_temperature',
                                           'minimum_temperature', 'maximum_temperature'])
dataset = dataset.dropna()

n_input = 14
idx = round(dataset.shape[0]/n_input,)
print(idx)
idx = idx*n_input
dataset = dataset.iloc[:idx,:]
print(dataset.shape)

# split into train and test
train, test = split_dataset(dataset.values,n_input)
# evaluate model and get scores
score, scores = evaluate_model(train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
#days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
#pyplot.plot(days, scores, marker='o', label='lstm')
#pyplot.show()

7
(98, 12)
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 14, 100)           45200     
_________________________________________________________________
lstm_20 (LSTM)               (None, 14, 64)            42240     
_________________________________________________________________
lstm_21 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 99,889
Trainable params: 99,889
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
 - 1s - loss: 619715428.2857 - mse: 619715456.0000 - mae: 20821.8750 - mape: 99.9977 - accura

IndexError: ignored

In [ ]:
print(train.shape)
print(test.shape)
actual.shape

(5, 14, 12)
(1, 14, 12)


NameError: ignored